# MNIST classifier tutorial

* MNIST database of handwritten digits is a popular dataset to demonstrate machine learning classifier training.
* In this tutorial, we train a basic Neural Network (NN) classifier using PyTorch.
* Once the basic NN classifier training workflow has been defined, we parallelize the mutually independent tasks using Covalent.
* This tutorial has two primary objectives.
* One goal is to show the ease with which a "normal" workflow can be adapted to a Covalent workflow.
* The second objective is to display the browser-based Covalent workflow tracking UI.

In [1]:
# !pip install cova
# !conda install pytorch torchvision -c pytorch -y

In [2]:
import covalent as ct

from pathlib import Path

import time
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# # Add global variables inside the workflow 
# n_epochs = 2
# batch_size_train = 64
# batch_size_test = 1000
# learning_rate = 0.01
# momentum = 0.5
# log_interval = 10

# random_seed = 1
# torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)
# torch.autograd.set_detect_anomaly(True)

# network = Net()

In [4]:
def data_loader(batch_size: int,
                train: bool,
                download: bool = True,
                shuffle: bool = True,
                data_dir: str = '~/data/mnist/',
                alpha: float = 0.1307,  # normalization parameter
                beta: float = 0.3081  # normalization parameter
                ) -> torch.utils.data.dataloader.DataLoader:
    """Data loader"""

    data_dir = Path(data_dir).expanduser()  # Get result director absolute path
    data_dir.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist    

    return torch.utils.data.DataLoader(
        torchvision.datasets.MNIST(
            data_dir,
            train=train,
            download=download,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((alpha,), (beta,))
            ])),
        batch_size=batch_size,
        shuffle=shuffle)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [6]:
def get_optimizer(network, lr, momentum):
    return optim.SGD(network.parameters(), lr, momentum)

In [7]:
def train(train_loader, train_losses, train_counter, network, optimizer, n_epochs: int, results_dir: str = '~/data/mnist/results/') -> None:
    results_dir = Path(results_dir).expanduser()  # Get result director absolute path
    results_dir.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist

    for epoch in range(1, n_epochs + 1):
        network.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = network(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                    f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
                train_losses.append(loss.item())
                train_counter.append(
                    (batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset)))
                torch.save(network.state_dict(), f'{results_dir}model.pth')
                torch.save(optimizer.state_dict(), f'{results_dir}optimizer.pth')

In [8]:
def test(test_losses, test_loader, network):
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(f'\nTest set: Avg. loss: {test_loss:.4f}, Accuracy: {correct}/'
          f'{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [9]:
def workflow():

    n_epochs = 2
    batch_size_train = 64
    batch_size_test = 1000
    learning_rate = 0.01
    momentum = 0.5
    log_interval = 10

    random_seed = 1
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)
    torch.autograd.set_detect_anomaly(True)

    network = Net()

    train_loader = data_loader(batch_size=batch_size_train, train=True)
    test_loader = data_loader(batch_size=batch_size_test, train=False)

    train_losses, train_counter, test_losses = [], [], []
    # test_counter = [i * len(train_loader.dataset) for i in range(n_epochs + 1)]

    test(test_losses=test_losses, test_loader=test_loader, network=net)
    for epoch in range(1, n_epochs + 1):
        train(train_loader=train_loader, 
            train_losses=train_losses, 
            train_counter=train_counter, 
            network=net,
            optimizer=get_optimizer(network, learning_rate, momentum),
            epoch=epoch)
    test(test_losses=test_losses, test_loader=test_loader, network=net)  

start = time.time() 
workflow()
end = time.time()
print(f"Execution time of normal workflow: {end - start}")

/var/folders/hn/dnjxkq7946j9wpxv1cy9pm2m0000gn/T/ipykernel_70432/3183543750.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/Users/faiyaz/opt/anaconda3/envs/covalent-pytorch-dev/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3211, Accuracy: 403/10000 (4%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.365674
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310861
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.278081
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.262210
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.290020
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.283870
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.237407
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.224197
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.204942
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.189945
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.018581
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.980645
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.853754
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.857393
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.677071
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.652027
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.613182
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.520675
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.283667
Train Epoch: 1 [12160/6

In [10]:
data_loader = ct.electron(data_loader)
train = ct.electron(train)
test = ct.electron(test)
get_optimizer = ct.electron(get_optimizer)
workflow = ct.lattice(workflow)

In [13]:
start = time.time()
dispatch_id = ct.dispatch(workflow)()
print(dispatch_id)
result = ct.get_result(dispatch_id=dispatch_id, wait=True)
print(result.result)
end = time.time()
print(f"Covalent workflow execution time: {end - start}")

3fb152ec-da1c-4149-a3c0-4bab582b44ae
None
Covalent workflow execution time: 37.76052188873291


In [12]:
print(result)


Lattice Result
status: COMPLETED
result: None
inputs: {'net': Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)}
error: None

start_time: 2022-02-22 16:57:02.776267+00:00
end_time: 2022-02-22 16:57:38.370618+00:00

results_dir: /Users/faiyaz/Code/covalent/doc/source/tutorials/machine_learning/results
dispatch_id: c71005ec-ceaf-4817-b1fb-7e3e3b3bd985

Node Outputs
------------
data_loader(0): <torch.utils.data.dataloader.DataLoader object at 0x7fe3e32db580>
:parameter:64(1): 64
:parameter:True(2): True
data_loader(3): <torch.utils.data.dataloader.DataLoader object at 0x7fe3d285cb50>
:parameter:1000(4): 1000
:parameter:False(5): False
test(6): None
:electron_list:(7): []
:parameter:Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1)

Triple check to ensure there is no plagiarism issues
References:
- https://nextjournal.com/gkoehler/pytorch-mnist
- https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627